In [ ]:
import json

In [1]:
import datasets
from datasets import load_dataset
from datasets import Dataset
from sklearn.model_selection import train_test_split

In [ ]:
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForMaskedLM

In [ ]:
model_name = 'cointegrated/rubert-tiny'

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=312, padding='max_length')

In [ ]:
with open('bashim.json', encoding="utf-8") as f:
    bashim = json.load(f)

In [ ]:
text_list = []
for i in range(len(bashim)):
    text_list.append(bashim[i]['quote_text'])

In [ ]:
bag = [item for sentence in text_list for item in sentence.split('.') if item != '']

In [ ]:
bag = list(filter(lambda x: x != '\n', bag))

In [ ]:
def clear_eng(sent):
    new_sent = sent
    for i in new_sent:
        if i in alphabet:
            new_sent = new_sent.replace(i, '')
        elif i in num:
            return ''
    if len(new_sent) == 0:
        return ''
    if new_sent[0] == ' ':
        return new_sent[1:] 
    else:
        return new_sent

In [ ]:
alphabet = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMOPQRSTUVWXYZ<>^[-]()_%*""\\\n'
num = '0123456789'
new_bag = []
for sent in bag:
    if len(sent) > 8:
        new_sent = clear_eng(sent)
        if new_sent != '':
            new_bag.append(new_sent)
bag = new_bag

In [ ]:
train_bash, test_bash = train_test_split(bag, test_size=0.15)

In [ ]:
train_dd = Dataset.from_dict({'text': train_bash})
test_dd = Dataset.from_dict({'text': test_bash})

In [ ]:
bashim_dd = datasets.DatasetDict({'train': train_dd, 'test': test_dd})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
tokenizer_bashim = bashim_dd.map(preprocess_function, batched=True)

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
def labels_on(example):
    example['labels'] = example['input_ids'].copy()
    return example

In [ ]:
tokenizer_bashim = tokenizer_bashim.map(labels_on, batched=True)

In [ ]:
model = AutoModelForMaskedLM.from_pretrained(model_name)

In [ ]:
model.to('cuda')

In [ ]:
training_args = TrainingArguments(
    output_dir='D:/results/',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.001
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenizer_bashim["train"],
    eval_dataset=tokenizer_bashim["test"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model('./model_bert')

In [ ]:
from transformers import pipeline

model_bert = AutoModelForMaskedLM.from_pretrained('./model_bert', local_files_only=True)

fill_mask = pipeline(
    "fill-mask",
    model=model_bert,
    tokenizer=model_name,
    top_k=5
)
fill_mask('[MASK]')